# Hands On - Aprendizado Federado aplicado à Internet das Coisas

**Notebook 2**: Criação de clientes no ambiente federado

O reconhecimento da atividade humana é uma área de pesquisa ativa e que possui um enorme potencial de benefício com o uso de aprendizado federado (FL), já que tais dados são normalmente privados e possuem informações sensíveis sobre os usuários.
Além disso, com FL também podemos desenvolver um modelo conjunto que consiga capturar a diversidade dos dados, algo que é extremamente difícil de ser coletado de forma individual.

Sob esse contexto, nesse tutorial vamos aprender como definir clientes para o treinamento federado de uma rede neural para auxilar no reconhecimento de atividades humanas (*Human Activity Recognition* - HAR) usando o framework de aprendizado federado
Flower em conjunto com a biblioteca de deep learning Pytorch.

### Dataset

Os dados serão particionados horizontalmente, assim os subconjuntos de treinamento e teste irão ser divididos em mini-batches (pequenos lotes) com base no número total de clientes.

Para isso, aplicaremos uma função auxiliar para carregar os dados e definir os conjuntos de treinamento e teste.
Nessa função, precisaremos dos seguintes parâmetros: 

* **data root (str)**: Diretório onde os datasets finais serão armazenados. 

* **train batch size (int)**: Tamanho do mini-batch usado nos dados de treinamento.

* **test batch size (int)**: Tamanho do mini-batch usado nos dados de teste. 

* **id (int)**: Client ID usado para selecionar uma partição específica. 

* **nb clients (int)**: Número total de clientes usados no treinamento.

In [1]:
#Carregando os dados
import flwr as fl
import torch

# Para o windows, usando o auxiliary.py ao invés do aux.py
# https://stackoverflow.com/questions/36225708/cannot-create-a-directory-named-aux-or-starting-by-aux-on-windows-8-1
# https://bugs.launchpad.net/tensortoolbox/+bug/1400505
import auxiliary as aux 

DATA_ROOT = "./data/pml-training.csv"

cid = 0
nb_clients = 3
train_batch_size = 64
test_batch_size = 64
epochs = 10

# Load data
train_loader, test_loader = aux.load_data(
        data_root = DATA_ROOT,
        train_batch_size = train_batch_size,
        test_batch_size = test_batch_size,
        cid = cid,
        nb_clients = nb_clients + 1,
)

### Rede Neural

Atualmente o modelo de classificação mais adequado e vantajoso para a modelagem de um ambiente federado são as redes neurais.
Definimos essa configuração de arquitetura por meio da criação de uma classe em Pytorch denominada **HARmodel** presente no arquivo auxiliar *aux.py (auxiliary.py)*  adicionado.

### Cliente Flower

O próximo passo é definir a alocação dos dispositivos no ambiente federado. 

Quando o servidor seleciona um dispositivo específico do ambiente federado para realizar um treinamento, ele envia as instruções pela rede, por meio de uma interface chamada **Client**.
Assim, o cliente recebe as instruções do servidor e chama um dos métodos desta classe para executar seu código (ou seja, para treinar a sua rede neural local). 

O framework Flower fornece uma classe chamada *NumPyClient*, que torna mais fácil implementar a interface do cliente quando utilizamos PyTorch. 
Quando implementamos um NumPyClient devemos definir os seguintes métodos: 

* **get_parameters**: retorna o peso do modelo
como uma lista de ndarrays 

* **set_parameters** (opcional): atualiza os pesos do modelo
local com os parâmetros recebidos do servidor 

* **fit**: define os pesos do modelo local, treina o modelo localmente e recebe o update dos pesos locais 

* **evaluate**: define como o modelo local será testado. 

Abaixo mostramos como a classe Client foi implementada
para o caso de estudo apresentado:

In [2]:
class FlowerClient(fl.client.Client):
    """Flower client implementing classification using PyTorch."""

    def __init__(self, cid, train_loader, test_loader, epochs, device: torch.device = torch.device("cpu")):
        
        self.model = HARmodel(40, 5).to(device)
        self.cid = cid
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device
        self.epochs = epochs

    def get_weights(self):
        """Get model weights as a list of NumPy ndarrays."""
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_weights(self, weights):
        """Set model weights from a list of NumPy ndarrays.
        Parameters
        ----------
        weights: fl.common.Weights
            Weights received by the server and set to local model
        Returns
        -------
        """
        state_dict = OrderedDict(
            {
                k: torch.Tensor(v)
                for k, v in zip(self.model.state_dict().keys(), weights)
            }
        )
        self.model.load_state_dict(state_dict, strict=True)

    def get_parameters(self):
        """Encapsulates the weights into Flower Parameters """
        weights: fl.common.Weights = self.get_weights()
        parameters = fl.common.weights_to_parameters(weights)
        return fl.common.ParametersRes(parameters=parameters)

    def fit(self, ins):
        """Trains the model on local dataset
        Parameters
        ----------
        ins: fl.common.FitIns
           Parameters sent by the server to be used during training.
        Returns
        -------
            Set of variables containing the new set of weights and information the client.
        """

        # Set the seed so we are sure to generate the same global batches
        # indices across all clients
        np.random.seed(123)

        weights: fl.common.Weights = fl.common.parameters_to_weights(ins.parameters)
        fit_begin = timeit.default_timer()

        # Set model parameters/weights
        self.set_weights(weights)

        # Train model
        num_examples_train: int = train(
            self.model, self.train_loader, epochs = self.epochs, device = self.device, cid = self.cid
        )

        # Return the refined weights and the number of examples used for training
        weights_prime: fl.common.Weights = self.get_weights()
        params_prime = fl.common.weights_to_parameters(weights_prime)
        fit_duration = timeit.default_timer() - fit_begin
        return fl.common.FitRes(
            parameters = params_prime,
            num_examples = num_examples_train,
            num_examples_ceil = num_examples_train,
            fit_duration = fit_duration,
        )

    def evaluate(self, ins):
        """
        Parameters
        ----------
        ins: fl.common.EvaluateIns
           Parameters sent by the server to be used during testing.
        Returns
        -------
            Information the clients testing results.
        """
        weights = fl.common.parameters_to_weights(ins.parameters)

        # Use provided weights to update the local model
        self.set_weights(weights)

        (num_examples_test, test_loss, accuracy) = test(self.model, self.test_loader, device = self.device)
        print(f"Client {self.cid} - Evaluate on {num_examples_test} samples: Average loss: {test_loss:.4f}, Accuracy: {100*accuracy:.2f}%\n")

        # Return the number of evaluation examples and the evaluation result (loss)
        return fl.common.EvaluateRes(
            num_examples = num_examples_test,
            loss = float(test_loss),
            accuracy = float(accuracy),
        )

### Instanciando o cliente

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

client = aux.FlowerClient(
    cid = cid,
    train_loader = train_loader,
    test_loader = test_loader,
    epochs = epochs,
    device = device,
)

### Inicializando o cliente

O flower nos fornece a possibilidade de rodar o servidor e o cliente na mesma máquina, configurando o endereço do servidor como "[::]: 8080". 
Porém, se quisermos implementar uma aplicação realmente federada com o servidor e clientes em execução em diferentes máquinas, precisaremos apenas alterar o server address para o respectivo endereço da máquina do cliente.

In [4]:
client_address = "[::]:8081"
fl.client.start_client(client_address, client)

DEBUG flower 2021-08-20 11:19:15,775 | connection.py:36 | ChannelConnectivity.IDLE
INFO flower 2021-08-20 11:19:15,785 | app.py:61 | Opened (insecure) gRPC connection
DEBUG flower 2021-08-20 11:19:15,799 | connection.py:36 | ChannelConnectivity.TRANSIENT_FAILURE
DEBUG flower 2021-08-20 11:19:15,993 | connection.py:68 | Insecure gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1629469155.786000000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3009,"referenced_errors":[{"created":"@1629469155.786000000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":398,"grpc_status":14}]}"
>